In [11]:
# Import
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
tf.enable_eager_execution()

1.13.1


In [12]:
# Load Data 
filename = 'C:\\Users\\deniz\\Desktop\\Thesis of ML for AutoPas\\Data\\Batch5\\final6.txt'
arr = np.genfromtxt(filename, delimiter=',')
print(arr.shape)

(81542, 539)


In [13]:
# Basic model with 4 paramters
parameter_count = 4

In [18]:
# Note, make sure the order of 
data = np.copy(arr)

# Shuffle data and take 50% as test data
np.random.shuffle(data)
test_size = (np.ceil(data.shape[0] / 2)).astype(int)
train_params = data[test_size:,0:parameter_count]
train_labels = np.argmin(data[test_size:,-23:], 1).astype(int)
test_params = data[:test_size,0:parameter_count]
test_labels = np.argmin(data[:test_size,-23:], 1).astype(int)


# Posible normalization functions
def normalize01(array):
    divisor = np.max(array)
    for i in range(array.size):
        array[i] = array[i] / divisor
        
np.apply_along_axis(normalize01, 0, train_params)
np.apply_along_axis(normalize01, 0, test_params)
print(train_params[:10])

[[0.33333333 0.5        0.078125   0.085184  ]
 [1.         0.75       0.015625   0.085184  ]
 [1.         1.         0.0625     0.074088  ]
 [1.         0.25       0.03125    0.039304  ]
 [1.         0.25       0.046875   0.054872  ]
 [0.66666667 0.25       0.046875   0.074088  ]
 [0.66666667 0.75       0.03125    0.216     ]
 [0.33333333 0.75       0.0625     0.013824  ]
 [0.33333333 0.75       0.125      0.884736  ]
 [0.33333333 1.         0.0625     0.175616  ]]


In [19]:
# Check how the total data is distributed among the labels
dist = np.bincount(np.concatenate((train_labels, test_labels)))
print(dist)
print(np.argmax(dist))
print(np.max(dist) / data.shape[0])

[ 4066  2771  4742  4312 34431     0     0   652    13    28    16   255
  1065   411     0    34    10    77     1    33 24520  3218   887]
4
0.42224865713374704


In [20]:
# Model
model = keras.Sequential([
    #keras.layers.Dense(parameter_count, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(4,)),
    keras.layers.Dense(23, activation=tf.nn.softmax)
])

opt = keras.optimizers.Adam(lr=0.001)

model.compile(optimizer=opt, 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
# Fun
model.fit(train_params, train_labels , epochs=50)

Epoch 1/50
40771/40771 [==============================] - 4s 93us/sample - loss: 1.5098 - acc: 0.5826
Epoch 2/50
40771/40771 [==============================] - 3s 64us/sample - loss: 1.0310 - acc: 0.7098
Epoch 3/50
40771/40771 [==============================] - 3s 64us/sample - loss: 0.9276 - acc: 0.7219
Epoch 4/50
40771/40771 [==============================] - 3s 65us/sample - loss: 0.8794 - acc: 0.7262
Epoch 5/50
40771/40771 [==============================] - 3s 75us/sample - loss: 0.8511 - acc: 0.7285
Epoch 6/50
40771/40771 [==============================] - 3s 66us/sample - loss: 0.8319 - acc: 0.7303
Epoch 7/50
40771/40771 [==============================] - 3s 69us/sample - loss: 0.8182 - acc: 0.7323
Epoch 8/50
40771/40771 [==============================] - 3s 70us/sample - loss: 0.8071 - acc: 0.7334
Epoch 9/50
40771/40771 [==============================] - 3s 70us/sample - loss: 0.7980 - acc: 0.7355
Epoch 10/50
40771/40771 [==============================] - 3s 74us/sample - loss: 

In [22]:
predictions = model.predict(test_params)
test_loss, test_acc = model.evaluate(test_params, test_labels)
print('Test accuracy:', test_acc)

40771/40771 [==============================] - 2s 46us/sample - loss: 0.7009 - acc: 0.7599
Test accuracy: 0.75987834


In [23]:
# Print prediction, result, and how certain the result is
best = np.argsort(predictions)
print(' Best  Guess1  Guess2  Certainty1      Certainty2')
for i in range(20):
    print(' ', test_labels[i], '\t', best[i][-1], '\t', best[i][-2], '\t', \
          "{:.2%}".format(predictions[i][best[i][-1]]), '\t', "{:.2%}".format(predictions[i][best[i][-2]]))

 Best  Guess1  Guess2  Certainty1      Certainty2
  4 	 4 	 3 	 72.45% 	 22.82%
  1 	 2 	 1 	 38.63% 	 37.39%
  0 	 0 	 20 	 56.01% 	 39.20%
  1 	 2 	 20 	 50.39% 	 13.01%
  21 	 4 	 3 	 36.98% 	 32.65%
  2 	 20 	 2 	 62.45% 	 10.52%
  1 	 20 	 0 	 33.27% 	 27.48%
  4 	 4 	 3 	 87.77% 	 12.15%
  4 	 4 	 3 	 99.25% 	 0.38%
  3 	 3 	 4 	 47.09% 	 37.13%
  4 	 4 	 1 	 98.75% 	 0.74%
  4 	 4 	 1 	 98.89% 	 0.58%
  4 	 4 	 3 	 98.62% 	 0.73%
  0 	 20 	 0 	 77.48% 	 17.08%
  20 	 20 	 0 	 74.29% 	 8.80%
  1 	 2 	 20 	 37.18% 	 20.67%
  4 	 4 	 3 	 90.92% 	 9.08%
  4 	 4 	 3 	 88.32% 	 11.66%
  20 	 20 	 0 	 90.93% 	 3.69%
  4 	 4 	 3 	 88.96% 	 10.80%


In [24]:
# Print general statistics about in how many guesses the AI would be correct
correct = np.zeros(best.shape[1])
most_occuring = np.sort(np.bincount(test_labels))[::-1]
for i in range(test_size):
    for j in range(correct.size):
        if best[i][-j-1] == test_labels[i]:
            correct[j] = correct[j] + 1
            break
np.set_printoptions(precision=3)
print('The count of guesses until correct choice:', correct.astype(int))
print('Cumilative chance that the choice was correct by:', \
      np.apply_along_axis(lambda x: x / test_size, 0, np.cumsum(correct))[0:5])
print('The count of most occuring tests:', most_occuring)
print('Cumilative chance that the choice was correct by:', \
      np.apply_along_axis(lambda x: x / test_size, 0, np.cumsum(most_occuring))[0:5])

The count of guesses until correct choice: [30981  5486  2516   715   444   324   183    47    19    31    19     4
     1     0     0     0     1     0     0     0     0     0     0]
Cumilative chance that the choice was correct by: [0.76  0.894 0.956 0.974 0.985]
The count of most occuring tests: [17246 12259  2374  2174  2038  1612  1388   509   427   317   205   120
    38    15    14    12     9     7     6     1     0     0     0]
Cumilative chance that the choice was correct by: [0.423 0.724 0.782 0.835 0.885]


In [25]:
# Display relative timing of all experiments, and print the ones which are relatively close
timings = np.apply_along_axis(lambda x: np.sort(x), 1, arr[:, parameter_count:])
for i in range(timings.shape[0]):
    fastest = timings[i][0]
    for j in range(timings.shape[1]):
        timings[i][j] = timings[i][j] / fastest

for i in range(3):
    print(timings[i])
    
print('The average:', np.average(timings, 0))

count = 0
for i in range(timings.shape[0]):
    if timings[i][1] < 1.01:
        count = count + 1
print(count, 'second best results from', timings.shape[0], 'are within 1 percent speed difference')

count = 0
for i in range(timings.shape[0]):
    for j in range(1, timings.shape[1]):
        if timings[i][j] < 1.01:
            count = count + 1
print(count, 'non best results from', timings.shape[0], 'are within 1 percent speed difference')


count = 0
for i in range(timings.shape[0]):
    if timings[i][1] < 1.05:
        count = count + 1
print(count, 'second best results from', timings.shape[0], 'are within 5 percent speed difference')

C:\Users\deniz\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\deniz\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  


[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan na

10996 second best results from 81542 are within 1 percent speed difference
24714 non best results from 81542 are within 1 percent speed difference
19060 second best results from 81542 are within 5 percent speed difference


In [26]:
# Count the first guesses that were relatively quick
test_timings = data[:test_size,parameter_count:]
for i in range(test_timings.shape[0]):
    fastest = np.min(test_timings[i])
    for j in range(test_timings.shape[1]):
        test_timings[i][j] = test_timings[i][j] / fastest
        
count = 0
for i in range(test_size):
    if test_timings[i][best[i][-1]] < 1.01:
        count = count + 1
print(count, 'first guesses from', test_size, 'are within 1 percent speed difference')

count = 0
for i in range(test_size):
    if test_timings[i][best[i][-1]] < 1.05:
        count = count + 1
print(count, 'first guesses from', test_size, 'are within 5 percent speed difference')

count = 0
for i in range(test_size):
    if test_timings[i][best[i][-1]] < 1.01 or test_timings[i][best[i][-2]] < 1.01:
        count = count + 1
print(count, 'of first two guesses from', test_size, 'are within 1 percent speed difference')

C:\Users\deniz\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\Users\deniz\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


119 first guesses from 40771 are within 1 percent speed difference
130 first guesses from 40771 are within 5 percent speed difference
1068 of first two guesses from 40771 are within 1 percent speed difference


In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 23)                391       
Total params: 471
Trainable params: 471
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.save('C:/Users/deniz/Desktop/Thesis of ML for AutoPas/keras_model.h5', include_optimizer=False) #include_optimizer=False